In [ ]:
import os
import mne
depth = 0.8
lambda2 = 0.1
recon_all_dir = r"D:\REFTEP_ALL\REFTEP_reco"
path_in_server = fr" "
os.mkdir(path_in_server)
for mri_dir in os.listdir(recon_all_dir):
    if 'fatsuppressed' not in mri_dir:
        subjects_mri_directory = os.path.join(recon_all_dir,mri_dir)
        site = mri_dir.split("_")[0]
        source_site = rf"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
        source_site_server = os.path.join(path_in_server,site)
        os.mkdir(source_site_server)
        for subject in os.listdir(subjects_mri_directory):
            if "sub-" not in subject:
                continue
            reftep_subject = subject.split("_reco")[0]
            subject_reftep_now = [name for name in os.listdir(source_site) if reftep_subject in name]
            if len(subject_reftep_now) not in [0,1]:
                print("Too many identifiers for this subject")
                break
            elif len(subject_reftep_now) == 0:
                print(f'did not find subject {reftep_subject} in {source_site}')
            else:
                reftep_subject = subject_reftep_now[0]
                subject_path_server = os.path.join(source_site_server,reftep_subject)
                os.mkdir(subject_path_server)
                sourcepath_subject = os.path.join(source_site,reftep_subject)
                subject_mri_dir = os.path.join(subjects_mri_directory,subject)#the subject mri directory in subjects_mri_directory
                epochs_file = os.path.join(sourcepath_subject,f'{reftep_subject}_final_eeg-epo.fif') #epochs file with digitization coordinates
                epochs = mne.read_epochs(epochs_file)
                transpath = os.path.join(sourcepath_subject,f'{reftep_subject}_coreg',f'{reftep_subject}-trans.fif') #eeg-mri coregistration transformation matrix
                bempath = os.path.join(subject_mri_dir, 'bem','bem-solution')
                bem_solution = mne.read_bem_solution(bempath) #load the bem solution
                fname_src = os.path.join(subjects_mri_directory,subject,f'{subject}-oct-6-src.fif')
                src = mne.read_source_spaces(fname_src) #read source spaces
                fwd_fname = os.path.join(sourcepath_subject, f'{reftep_subject}-fwd.fif')
                fwd = mne.read_forward_solution(fwd_fname)
                cov_fname = os.path.join(sourcepath_subject, f'{reftep_subject}_adhoc-cov.fif')
                cov = mne.read_cov(cov_fname)

                inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, cov, loose=0, depth=depth, fixed=True) #create the inverse operator W
                stcs = mne.minimum_norm.apply_inverse_epochs(epochs, inv, lambda2=lambda2, method='MNE') #operate on y with W to get the source space signals
                for ind, stc in enumerate(stcs):
                    stc_filename = os.path.join(subject_path_server,f'{reftep_subject}-epoch_{ind}')
                    stc.save(stc_filename)
                del stcs

                    

